<img src="../images/Lektion6.png" style="margin: 20px auto 20px 0px"/>
<h2 style="display:none">Lektion 6 - Funktionen höherer Ordnung</h2>

Ein weiteres Konzept der funktionalen Programmierung sind Funktionen höherer Ordnung. Das Charakteristische an ihnen ist, dass ihnen eine weitere Funktion übergeben wird. Ermöglicht wird dies dadurch, dass Funktionen als Objekte angesehen werden. Die bis jetzt verwendeten Funktionen werden, im Gegensatz dazu, Funktionen erster Ordnung genannt.  
Anwendung findet diese Art der Funktion meist, wenn um eine Funktion herum noch etwas passieren soll. Gebräuchliche Beispiele sind die Zeitmessung, bei der die höhere Funktion lediglich die übergebene Funktion mit Messfunktionen einrahmt, oder auch beim Schreiben einer Datei. Die übergebene Funktion wird als Lambda übergeben. Schauen wir uns das Beispiel der Zeitmessung genauer an:

In [1]:
fun messeZeit(f: () -> Any): Long {
    val start = System.currentTimeMillis()
    f()
    val ende = System.currentTimeMillis()
    return ende - start
}

fun fibonacci(n: Int, a: Long, b: Long): Long {
    return if (n == 0) b else fibonacci(n-1, a+b, a)
}

fun fakultaet(n : Int): Int{
    return if (n == 1) 1 else n * fakultaet(n - 1) 
}

println("Dauer von fibonacci(5000, 1000, 1000): ${messeZeit { fibonacci(5000, 1000, 1000) }} ms")

Dauer von fibonacci(5000, 1000, 1000): 1 ms


In [ ]:
println("Dauer von fakultaet(10000): ${messeZeit { fakultaet(10000) }} ms")

Die Funktion höherer Ordnung `messeZeit` bekommt eine Funktion (Rückgabetyp `Any`) übergeben. Im Rumpf wird die Systemzeit vor und nach dem Ausführen der übergebenen Funktion gemessen.  
Auch können Lambdas direkt übergeben werden:

In [3]:
fun subtrahiere (f1: () -> Int, f2: () -> Int) : Int{
    return f1() - f2()
}

subtrahiere ( { 5 * 10 }, { 3 * 3 } )

41

Weiterführen lässt sich dieses Beispiel mit der Übergabe eines weiteren Parameters an die Lambda-Ausdrücke:

In [4]:
fun subtrahiere (f1: (n: Int) -> Int, f2: (m: Int) -> Int, x: Int) : Int{
    return f1(x) - f2(x)
}

subtrahiere ( { x -> x * 10 }, { it * 3 }, 5 ) //50-15=35

35

Es endet aber nicht damit, dass die Funktionen höherer Ordnung nur Funktionen als Parameter übergeben bekommen können. Ebenso ist eine Rückgabe einer Funktion möglich.

In [5]:
fun addiere5(): (Int) -> Int = {n -> n + 5}
println("addiere5 gibt eine Funktion zurück: ${addiere5()}")

addiere5 gibt eine Funktion zurück: (kotlin.Int) -> kotlin.Int


Die Funktion höherer Ordnung `addiere5` hat selbst keinen Parameter. Der Rückgabetyp ist hier aber *unnormal*. Anstelle einer einfachen Angabe (zum Beispiel `Int`), wird ein Lambda-ähnlicher Ausdruck zurückgegeben. Dieser beschreibt den Typ der zurückgegeben Funktion. Dieser wird ein Parameter vom Typ `Int` übergeben und liefert als Ergebnis auch einen Wert vom Typ `Int`. Daraufhin ist in einem Lambda-Ausdruck der Rumpf der Funktion beschrieben. Der Parameter vom Typ `Int` wird *n* genannt (da nur ein Parameter vorhanden ist, könnte dieser auch mit `it` angesprochen werden). Nach einem `->` folgt der eigentliche Rumpf. In diesem Fall wird zu dem Parameter 5 addiert und das Ergebnis zurückgegeben.  
Wird nun die Funktion `addiere5` ganz normal aufgerufen, wird die definierte Funktion zurückgegeben. Soll diese aber ausgeführt werden, ist ein weiterer Funktionsaufruf mit den benötigten Parametern auszuführen.

In [6]:
addiere5()(10)
println("addiere5()(10): ${addiere5()(10)}")

addiere5()(10): 15


Zusätzlich kann der Funktion der höheren Ordnung auch ein Parameter übergeben werden. In dem oben genannten Beispiel kann dann beispielsweise der bis jetzt feste Wert von 5 variabel gesetzt werden. Mit der Funktion `addiereN` besteht die Möglichkeit beliebige Funktionen zu generieren.

In [7]:
fun addiereN(m: Int): (Int) -> Int = {n -> n + m}
val addiere10 = addiereN(10)
println("addiere10: ${addiere10}")
println("addiere10(20): ${addiere10(20)}")
val addiere50 = addiereN(50)
println("addiere50: ${addiere50}")
println("addiere50(20): ${addiere50(20)}")

addiere10: (kotlin.Int) -> kotlin.Int
addiere10(20): 30
addiere50: (kotlin.Int) -> kotlin.Int
addiere50(20): 70


### inline-Funktionen
Bisher wird in den Funktionen höherer Ordnung die übergebene Funktion aufgerufen. Das heißt, es muss Arbeitsspeicher reserviert, Parameter und lokale Variablen zwischengespeichert werden. Dies kann dazu führen, dass die Schnelligkeit leidet. Für dieses Problem gibt es aber eine einfache Lösung: `inline`-Funktionen. Diese sind im Grunde Funktionen höherer Ordnung, aber kopieren den Code der übergebenen Funktion an die dafür vorgesehenen Stellen. Es wird also nicht direkt die Funktion aufgerufen, sondern diese in den Funktionsrumpf der höheren Funktion integriert. Gekennzeichnet wird dies mit dem Zusatz `inline` zu Beginn des Funktionskopf.

In [8]:
inline fun subtrahiereInline (f1: (n: Int) -> Int, f2: (m: Int) -> Int, x: Int) : Int{
    return f1(x) - f2(x)
}

//Im Hintergrund wird der Code von f1 und f2 in subtrahiereInline kopiert. Für diesen konkreten Anwendungsfall:
fun subtrahiereInlineKopiert (f1: (n: Int) -> Int, f2: (m: Int) -> Int, x: Int) : Int{
    return (x * 10) - (x * 3)
}

subtrahiereInline ( { x -> x * 10 }, { it * 3 }, 5 ) //50-15=35

35